# Q&A over Documents
One of the common and complex application people are building using an LLM is a system that can answer questions on top of or about a document. So, given a piece of text, maybe extracted from a PDF file or from a webpade or from some company's intranet internal document collection, can we use an LLM to answer questions about the content of those documents to help users gain a deeper understanding and get access to the information that they need? This is really powerful because it starts to combine these language models with data that they weren't originally trained on. So it makes them much more flexible and adaptable to our use case.

In [1]:
#pip install --upgrade langchain

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI

In [5]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [6]:
from langchain.indexes import VectorstoreIndexCreator

In [7]:
#pip install docarray

In [8]:
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])

In [9]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

**Note**:
- The notebook uses `langchain==0.0.179` and `openai==0.27.7`
- For these library versions, `VectorstoreIndexCreator` uses `text-davinci-003` as the base model, which has been deprecated since 1 January 2024.
- The replacement model, `gpt-3.5-turbo-instruct` will be used instead for the `query`.
- The `response` format might be different than the video because of this replacement model.

In [10]:
llm_replacement_model = OpenAI(temperature=0, model='gpt-3.5-turbo-instruct')

response = index.query(query, llm = llm_replacement_model)

In [11]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rated, wrinkle-resistant, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rated, SunSmart technology, wrinkle-free, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rated, wrinkle-resistant, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rated, moisture-wicking, fits comfortably over swimsuit, abrasion-resistant, imported | SPF

Language models can only inspect a few thousand words at a time. If we have really large documents then we use **embeddings** and **vector stores**.

### Embeddings
Embeddings create numerical representations for pieces of text. This numerical representation captures the semantic meaning of the piece of text that it's been run over. Pieces of text with similar content will have similar vectors. This lets us compare pieces of text in the vector space. This will let us easily figure out which text are like each other, which will be very useful as we think of which piece of text are like each other, which will be very useful as we think about which pieces of text we want to include when passing to the language model to answer a question.

### Vector Database
A vector database is a way to store these vector representations that we created in the previous step. We populate these vectors with chunks of text coming from incoming documents. When we get a big incoming document, we are first going to break it up into smaller chunks. This helps create piece of text that are smaller than the original document, which is useful because we may not be able to pass the whole document to the language model. So we want to create these small chunks so we can only pass the most relevant ones to the model. We then create an embedding for each of these chunks, and then we sotre those in a vector database. That's what happens when we create the index. We can use this index during runtime to find the pieces of text most relevant to an incoming query. When a query comes in, we first create an embedding for that query. We then compare it to all the vectors in the vector database, and we pick the **n** most similar. These are then returend, and we can pass those in the prompt to the language model to get back a final answer.

## Step By Step

In [12]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [13]:
docs = loader.load()

In [14]:
docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

#### Since these documents are small we don't need to create chunks we can direct start creating embeddings.

In [15]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [16]:
embed = embeddings.embed_query("Hi my name is Ebad")

In [17]:
print(len(embed))

1536


In [18]:
print(embed[:5])

[-0.02199380286037922, 0.006747527979314327, -0.018252847716212273, -0.039167046546936035, -0.013997197151184082]


In [19]:
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

In [20]:
query = "Please suggest a shirt with sunblocking"

In [21]:
docs = db.similarity_search(query)

In [22]:
len(docs)

4

Returns a list of 4 documents.

In [23]:
docs[0]

Document(page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255})

In [24]:
retriever = db.as_retriever()

To use this to do QnA we need to create a **retriever** from this vector store. A retriever is a generic interface that can be underpinned by any method that takes in a query and returns documents. Vecotr stores and embeddings are one such method to do so, although there are plenty of different methods, some less advanced, some more advanced.<br>
The retriever we created is just an interface for fetching documents. It is used to fetch the documents and pass it to the model.

In [25]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [26]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [27]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

In [28]:
display(Markdown(response))

| Name | Description |
| --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ sun protection, moisture-wicking, and abrasion-resistant fabric. Recommended by The Skin Cancer Foundation. |
| Men's Plaid Tropic Shirt | Ultracomfortable shirt with UPF 50+ sun protection, wrinkle-free fabric, and front/back cape venting. Made with 52% polyester and 48% nylon. |
| Men's TropicVibe Shirt | Men's sun-protection shirt with built-in UPF 50+ and front/back cape venting. Made with 71% nylon and 29% polyester. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+ sun protection, front/back cape venting, and two front bellows pockets. Made with 100% polyester. |

All of these shirts provide UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They also have additional features such as moisture-wicking, wrinkle-free fabric, and front/back cape venting for added comfort.

All of the above steps can be encapsulated with the langchain's Chain method.

In [29]:
qa_stuff = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, verbose=True)

In [30]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [31]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [32]:
display(Markdown(response))

| Shirt Number | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | This shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | This shirt is made with 52% polyester and 48% nylon. It is machine washable and dryable. It has front and back cape venting, two front bellows pockets, and is rated to UPF 50+. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | This shirt is made of 71% Nylon and 29% Polyester. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays. |
| 255 | Sun Shield Shirt | This shirt is made of 78% nylon and 22% Lycra Xtra Life fiber. It is handwashable and line dry. It is rated UPF 50+ for superior protection from the sun's UV rays. It is abrasion-resistant and wicks moisture for quick-drying comfort. |

The Men's Tropical Plaid Short-Sleeve Shirt is made of 100% polyester and is wrinkle-resistant. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays.

The Men's Plaid Tropic Shirt, Short-Sleeve is made with 52% polyester and 48% nylon. It has front and back cape venting, two front bellows pockets, and is rated to UPF 50+.

The Men's TropicVibe Shirt, Short-Sleeve is made of 71% Nylon and 29% Polyester. It has front and back cape venting that lets in cool breezes and two front bellows pockets. It is rated UPF 50+ for superior protection from the sun's UV rays.

The Sun Shield Shirt is made of 78% nylon and 22% Lycra Xtra Life fiber. It is abrasion-resistant and wicks moisture for quick-drying comfort. It is rated UPF 50+ for superior protection from the sun's UV rays. It is handwashable and line dry.

In [33]:
response = index.query(query, llm=llm)

In [34]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

## Stuff Method
This method is the simplest method. We simply stuff all data into the prompt as context to pass to the language model.
- **Pros:** It makes a single call to the LLM. The LLM has access to all the data at once.
- **Cons:** LLMs have a context length, and for large documetns or many documents this will not work as it will result in a prompt larger than the context length.

## Additional Methods
<img src="map_reduce.png" width="800" height="300"><br>
This basically takes all the chunks, passe them along with the question to a language model, gets back a response, and then uses another language model call to summarize all of the individual responses into a final answer. This is really powerful because it can operate over any number of documents and we can do it with individual questions in parallel. But it does takes a lot more calls. And it does treat all the documents as independent, which may not always be the most desired thing.

<img src="refine.png" width="800" height="300"><br>
This is again used to loop over many documents. But it actually does it intreatively. It builds upon the answer from the previous document. So this is really good for combining information and building up an answer over time. It will generally lead to longer answers and it is also not as fast because now the calls aren't independent. They depend on the result of previous calls. This means that it often takes a good while longer and takes just as amny calls as "Map_reduce".

<img src="map_rerank.png" width="800" height="300"><br>
This is pretty intresting and more experimental one where we do a single call to the language model for each document. And we also ask it to return a score. And then we select the highest score. This relies on the language model to know what the score should be. So we often have to tell it, "it should be a high score of it's relevant to the document and really refine the instructions there". Similar to "Map_reduce", all the calls are independent. So we can batch them and it is relatively fast. But again here we are making a bunch of langauge model calls. So it will be more expensive.